In [1]:
import ray
import numpy as np

In [2]:
ray.init(num_cpus=4)

RayContext(dashboard_url='', python_version='3.9.13', ray_version='1.13.0', ray_commit='e4ce38d001dbbe09cd21c497fedd03d692b2be3e', address_info={'node_ip_address': '127.0.0.1', 'raylet_ip_address': '127.0.0.1', 'redis_address': None, 'object_store_address': '/tmp/ray/session_2022-09-19_23-14-09_190176_48428/sockets/plasma_store', 'raylet_socket_name': '/tmp/ray/session_2022-09-19_23-14-09_190176_48428/sockets/raylet', 'webui_url': '', 'session_dir': '/tmp/ray/session_2022-09-19_23-14-09_190176_48428', 'metrics_export_port': 55004, 'gcs_address': '127.0.0.1:57945', 'address': '127.0.0.1:57945', 'node_id': '1f25b3c281a373247449d91f75e717856cae60ae4c0132cad70a6cd2'})

In [37]:
@ray.remote
class Collection():
    def __init__(self):
        self.empty_list = []
    
    def add(self, item):
        self.empty_list.append(item)

    def show(self):
        return self.empty_list

In [38]:
@ray.remote
class PTSwap():
    def __init__(self, thing1, thing2):
        self.thing1 = thing1
        self.thing2 = thing2

    def show_things(self, collection):
        a = ray.put(self.thing1)
        collection.add.remote(a)
        return self.thing1, self.thing2

In [39]:
collection = Collection.remote()

In [40]:
actor = PTSwap.remote(1, 2)
actor2 = PTSwap.remote(3, 4)

In [41]:
actors = [actor, actor2]

In [42]:
for actor in actors:
    res = ray.get(actor.show_things.remote(collection))
    print(res)

(1, 2)
(3, 4)


In [45]:
ray.get(collection.show.remote())

[1, 3]

In [11]:
__, __, a = ray.get(actor.show_things.remote())

In [12]:
b = ray.get(a)

In [13]:
b

1

In [34]:
ray.put(actor.thing1)

AttributeError: 'ActorHandle' object has no attribute 'thing1'

In [30]:
actor = PTSwap.remote(1, 2)

In [31]:
ray.get(actor.show_things.remote())

(1, 2)

In [26]:
ray.shutdown()

In [7]:
a = ray.put(1)

In [8]:
b = ray.get(a)

In [9]:
b

1

In [10]:
from ray.util import collective

2022-09-19 21:27:32,338	WARNING collective.py:19 -- NCCL seems unavailable. Please install Cupy following the guide at: https://docs.cupy.dev/en/stable/install.html.


In [14]:
b = collective.broadcast(np.array([1, 2, 3]))

RuntimeError: The collective APIs shall be only used inside a Ray actor or task.

In [15]:
ray.shutdown()